In [1]:
import pandas as pd
import numpy as np

In [2]:
Posts = pd.read_csv("travel_stackexchange/Posts.csv.gz", compression = 'gzip')
Users = pd.read_csv("travel_stackexchange/Users.csv.gz", compression = 'gzip')
Comments = pd.read_csv("travel_stackexchange/Comments.csv.gz", compression = 'gzip')
Badges = pd.read_csv("travel_stackexchange/Badges.csv.gz", compression = 'gzip')

In [3]:
import os, os.path
import sqlite3

In [4]:
baza = os.path.join('example.db')
if os.path.isfile(baza):
    os.remove(baza)

In [5]:
conn = sqlite3.connect(baza)

Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Comments.to_sql("Comments", conn)
Badges.to_sql("Badges", conn)

108812

In [6]:
# own function to comparing dataframes since method "equals" from pandas did not cooperate
def my_equals(df1, df2):
    if type(df1)!=type(df2):
        return False
    if df1.columns.equals(df2.columns) == False:
        return False
    if df1.dtypes.equals(df2.dtypes) == False:
        return False
    if df1.shape != df2.shape:
        return False
    
    values_comparision = df1.values == df2.values
    for row in values_comparision:
        if not all(cell for cell in row):
            return False
        
    return True

# First query

### pandas approach

In [7]:
from datetime import datetime
YearsOfPosts = Posts['CreationDate'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f').year)
YearsOfPosts = YearsOfPosts.to_frame()
YearsOfPosts['tmp'] = 1
result = YearsOfPosts.groupby('CreationDate')['tmp'].sum().reset_index()
result.columns = ['Year', 'TotalNumber']
resultPandas1=result

### SQL approach

In [8]:
resultSQL1 = pd.read_sql_query("""SELECT STRFTIME('%Y', CreationDate) AS Year, COUNT(*) AS TotalNumber
FROM Posts
GROUP BY Year""", conn)
resultSQL1['Year'] = resultSQL1['Year'].astype(int) # changing type to enable comparing

### Results comparision

In [9]:
print(my_equals(resultPandas1, resultSQL1))

True


# Second query

### pandas approach

In [10]:
questions = Posts[Posts['PostTypeId']==1].groupby('OwnerUserId')['ViewCount'].sum().reset_index()
questions.columns = ['OwnerUserId', 'TotalViews']
result = pd.merge(questions, Users, left_on='OwnerUserId', right_on='Id')
result = result[['Id', 'DisplayName', 'TotalViews']]
resultPandas2 = result.sort_values('TotalViews', ascending=False).head(10)

### SQL approach

In [11]:
resultSQL2 = pd.read_sql_query("""SELECT Id, DisplayName, TotalViews
FROM (
SELECT OwnerUserId, SUM(ViewCount) as TotalViews
FROM Posts
WHERE PostTypeId = 1
GROUP BY OwnerUserId
) AS Questions
JOIN Users
ON Users.Id = Questions.OwnerUserId
ORDER BY TotalViews DESC
LIMIT 10""", conn)

### Results comparision

In [12]:
print(my_equals(resultPandas2, resultSQL2))

True


# Third query

### pandas approach

In [13]:
questions = Posts[Posts['PostTypeId'] == 1].groupby('OwnerUserId').size().reset_index(name='QuestionsNumber')
answers = Posts[Posts['PostTypeId'] == 2].groupby('OwnerUserId').size().reset_index(name='AnswersNumber')
result = pd.merge(questions, answers, on='OwnerUserId')
result = result[result['AnswersNumber'] > result['QuestionsNumber']]
result = pd.merge(result, Users, left_on='OwnerUserId', right_on='Id')
result = result[['DisplayName', 'QuestionsNumber', 'AnswersNumber', 'Location', 'Reputation', 'UpVotes',
                 'DownVotes']].sort_values('AnswersNumber', ascending=False).head(5)
result['Location'] = result['Location'].fillna('None')
resultPandas3 = result
resultPandas3['Location'] = resultPandas3['Location'].astype(str)

### SQL approach

In [14]:
resultSQL3 = pd.read_sql_query("""SELECT DisplayName, QuestionsNumber, AnswersNumber,
Location,
Reputation, UpVotes, DownVotes
FROM (
SELECT *
FROM (
SELECT COUNT(*) as AnswersNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 2
GROUP BY OwnerUserId
) AS Answers
JOIN
(
SELECT COUNT(*) as QuestionsNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 1
GROUP BY OwnerUserId
) AS Questions
ON Answers.OwnerUserId = Questions.OwnerUserId
WHERE AnswersNumber > QuestionsNumber
ORDER BY AnswersNumber DESC
LIMIT 5
) AS PostsCounts
JOIN Users
ON PostsCounts.OwnerUserId = Users.Id""", conn)
resultSQL3['Location'] = resultSQL3['Location'].astype(str) # changing type to enable comparing

### Results comparision

In [15]:
print(my_equals(resultPandas3, resultSQL3))

True


# Fourth query

### pandas approach

In [ ]:
CmtTotScr = Comments.groupby('PostId')['Score'].sum().reset_index(name='CommentsTotalScore')
questions = Posts[Posts['PostTypeId'] == 1]
questions_with_comments = pd.merge(questions, CmtTotScr, left_on='Id', right_on='PostId')
result = pd.merge(questions_with_comments, Users, left_on='OwnerUserId', right_on='Id')
result = result[['Title', 'CommentCount', 'ViewCount', 'CommentsTotalScore', 'DisplayName', 'Reputation', 'Location']]
result = result.sort_values('CommentsTotalScore', ascending=False).head(10)
resultPandas4 = result
resultPandas4['Location'] = resultPandas4['Location'].fillna('None')
resultPandas4['Location'] = resultPandas4['Location'].astype(str)

### SQL approach

In [ ]:
resultSQL4 = pd.read_sql_query("""SELECT Title, CommentCount, ViewCount, CommentsTotalScore,
DisplayName, Reputation, Location
FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId
WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10""", conn)
resultSQL4['Location'] = resultSQL4['Location'].astype(str) # changing type to enable comparing

### Results comparision

In [ ]:
print(my_equals(resultPandas4, resultSQL4))

# Fifth query

### pandas approach

In [ ]:
from datetime import datetime
Badges['Year'] = Badges['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f').year)
badges_by_year_name = Badges.groupby(["Year", "Name"])["Name"].count().reset_index(name="Count")
badges_yearly_total = Badges.groupby("Year")["Name"].count().reset_index(name="CountTotal")
badges_merged = badges_by_year_name.merge(badges_yearly_total, on="Year")
badges_merged["MaxPercentage"] = badges_merged["Count"] / badges_merged["CountTotal"]
filt = badges_merged.groupby('Year')['MaxPercentage'].idxmax()
badges_result = badges_merged.loc[filt, ['Year', 'Name', 'MaxPercentage']]
resultPandas5 = badges_result

### SQL approach

In [ ]:
resultSQL5 = pd.read_sql_query("""SELECT Year, Name, MAX((Count * 1.0) / CountTotal) AS MaxPercentage
FROM (
SELECT BadgesNames.Year, BadgesNames.Name, BadgesNames.Count, BadgesYearly.CountTotal
FROM (
SELECT Name, COUNT(*) AS Count, STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
GROUP BY Name, Year
) AS BadgesNames
JOIN (
SELECT COUNT(*) AS CountTotal, STRFTIME('%Y', Badges.Date) AS Year
FROM Badges
GROUP BY YEAR
) AS BadgesYearly
ON BadgesNames.Year = BadgesYearly.Year
)
GROUP BY Year""", conn)
resultSQL5['Year'] = resultSQL5['Year'].astype(int) # changing type to enable comparing

### Results comparision

In [ ]:
print(my_equals(resultPandas5, resultSQL5))

In [ ]:
conn.close()